In [ ]:
import gpxpy
import gpxpy.gpx
import pandas as pd
from pathlib import Path
from typing import List, Tuple
from haversine import haversine, Unit
import folium
from folium.plugins import MarkerCluster
import os


# Step 1: 去除時間
def remove_time(gpx_file: str) -> gpxpy.gpx.GPX:
    """從 GPX 檔案中移除所有軌跡點的時間戳記。"""
    with open(gpx_file, "r", encoding="utf-8") as f:
        gpx = gpxpy.parse(f)
    for track in gpx.tracks:
        for segment in track.segments:
            for point in segment.points:
                point.time = None
    print(f"    - 已移除時間戳記")
    return gpx


# Step 2: 讀取通訊點
def load_communication_points(txt_file: str) -> List[Tuple[str, float, float, float]]:
    """從 TXT 檔案讀取通訊點資料。"""
    df = pd.read_csv(txt_file, sep="\t")
    points = []
    for _, row in df.iterrows():
        try:
            lat = float(str(row["緯度"]).replace("°", "").strip())
            lon = float(str(row["經度"]).replace("°", "").strip())
            ele = float(str(row["海拔（約）"]).replace("m", "").strip())
            label = f"{row['步道名稱']} {row['路標指示']}"
            points.append((label, lat, lon, ele))
        except (ValueError, KeyError) as e:
            print(f"      - ⚠️ 讀取通訊點時發生錯誤，請檢查 TXT 檔案格式：{e}")
            continue
    print(f"    - 成功讀取 {len(points)} 個通訊點")
    return points


# Step 2-2: 插入通訊點
def insert_points(
    gpx: gpxpy.gpx.GPX, comm_pts: List[Tuple[str, float, float, float]]
) -> gpxpy.gpx.GPX:
    """將通訊點插入到 GPX 軌跡中最接近的位置。"""
    if not gpx.tracks or not gpx.tracks[0].segments:
        print("    - ⚠️ GPX 檔案中沒有軌跡資料，無法插入通訊點。")
        return gpx

    segment = gpx.tracks[0].segments[0]
    points = segment.points

    for label, lat, lon, ele in comm_pts:
        closest_dist = float("inf")
        insert_idx = None
        # 尋找最佳插入點
        for i in range(len(points) - 1):
            p1 = (points[i].latitude, points[i].longitude)
            p2 = (points[i + 1].latitude, points[i + 1].longitude)
            comm_p = (lat, lon)

            # 計算點到線段的距離（簡化為點到兩端點距離和）
            dist_sum = haversine(p1, comm_p, unit=Unit.METERS) + haversine(
                p2, comm_p, unit=Unit.METERS
            )

            if dist_sum < closest_dist:
                closest_dist = dist_sum
                insert_idx = i + 1

        if insert_idx is not None:
            new_point = gpxpy.gpx.GPXTrackPoint(
                latitude=lat, longitude=lon, elevation=ele
            )
            segment.points.insert(insert_idx, new_point)
            # print(f"    - 已插入通訊點：{label}")
    print(f"    - 已將所有通訊點插入軌跡")
    return gpx


# Step 3: 切割通訊點之間路線
def split_single_segment(
    gpx: gpxpy.gpx.GPX,
    start_latlon: Tuple[float, float],
    end_latlon: Tuple[float, float],
) -> gpxpy.gpx.GPX:
    """根據起點和終點的經緯度切割 GPX 軌跡。"""
    new_gpx = gpxpy.gpx.GPX()
    new_track = gpxpy.gpx.GPXTrack()
    new_segment = gpxpy.gpx.GPXTrackSegment()
    new_gpx.tracks.append(new_track)
    new_track.segments.append(new_segment)

    if not gpx.tracks or not gpx.tracks[0].segments:
        print("    - ⚠️ GPX 檔案中沒有軌跡資料，無法進行切割。")
        return new_gpx

    points = gpx.tracks[0].segments[0].points

    def find_idx(target: Tuple[float, float]) -> int:
        return min(
            range(len(points)),
            key=lambda i: haversine(
                (points[i].latitude, points[i].longitude), target, unit=Unit.METERS
            ),
        )

    idx_start = find_idx(start_latlon)
    idx_end = find_idx(end_latlon)

    if idx_start > idx_end:
        idx_start, idx_end = idx_end, idx_start

    new_segment.points = points[idx_start : idx_end + 1]
    print(f"    - 路線切割完成：從索引 {idx_start} 至 {idx_end}")
    return new_gpx


# Step 4: 匯出 GPX 與 CSV
def export_gpx_and_csv(gpx: gpxpy.gpx.GPX, gpx_name: str, csv_name: str):
    """將處理後的 GPX 物件儲存為 GPX 和 CSV 檔案。"""
    # 儲存 GPX
    with open(gpx_name, "w", encoding="utf-8") as f:
        f.write(gpx.to_xml())
    print(f"    - ✅ 儲存 GPX：{gpx_name}")

    # 匯出 CSV
    data = []
    if gpx.tracks and gpx.tracks[0].segments:
        for pt in gpx.tracks[0].segments[0].points:
            data.append(
                {
                    "latitude": pt.latitude,
                    "longitude": pt.longitude,
                    "elevation": pt.elevation,
                }
            )
    df = pd.DataFrame(data)
    df.to_csv(csv_name, index=False)
    print(f"    - ✅ 匯出 CSV：{csv_name}")


# Step 5: 顯示地圖（使用內建 LayerControl）
def visualize_with_checkbox(original_gpx, sliced_gpx, comm_pts, output_html):
    """產生包含原始路徑、切割路徑和通訊點的互動式地圖。"""
    # 找到地圖中心點
    if (
        original_gpx.tracks
        and original_gpx.tracks[0].segments
        and original_gpx.tracks[0].segments[0].points
    ):
        first_point = original_gpx.tracks[0].segments[0].points[0]
        map_location = [first_point.latitude, first_point.longitude]
    else:
        map_location = [23.5, 121]  # 如果沒有點，預設台灣中心

    m = folium.Map(location=map_location, zoom_start=14)

    # --- 建立圖層 ---
    original_layer = folium.FeatureGroup(name="原始路線", show=True)
    cut_layer = folium.FeatureGroup(name="切割後路線", show=True)
    comm_layer = folium.FeatureGroup(name="通訊點", show=True)

    # --- 將資料加入對應圖層 ---
    # 原始路線
    if original_gpx.tracks and original_gpx.tracks[0].segments:
        original_coords = [
            (pt.latitude, pt.longitude)
            for pt in original_gpx.tracks[0].segments[0].points
        ]
        folium.PolyLine(original_coords, color="blue", weight=4, opacity=0.6).add_to(
            original_layer
        )

    # 切割後路線
    if sliced_gpx.tracks and sliced_gpx.tracks[0].segments:
        cut_coords = [
            (pt.latitude, pt.longitude)
            for pt in sliced_gpx.tracks[0].segments[0].points
        ]
        folium.PolyLine(cut_coords, color="red", weight=4, opacity=0.8).add_to(
            cut_layer
        )

    # 通訊點 (使用 MarkerCluster)
    cluster = MarkerCluster().add_to(comm_layer)
    for label, lat, lon, ele in comm_pts:
        folium.Marker(
            [lat, lon],
            popup=f"{label}<br>海拔：約{ele} m",
            icon=folium.Icon(color="purple"),
        ).add_to(cluster)

    # --- 將圖層加入地圖 ---
    original_layer.add_to(m)
    cut_layer.add_to(m)
    comm_layer.add_to(m)

    # --- 加入圖層控制器 (這會自動產生 Checkbox) ---
    folium.LayerControl().add_to(m)

    m.save(output_html)
    print(f"    - ✅ 地圖已儲存：{output_html}")


if __name__ == "__main__":
    # 🔹設定資料夾路徑
    gpx_folder = Path("hikingbook_route")
    txt_folder = Path("txt_data")
    gpx_output_folder = Path("gpx_data")
    csv_output_folder = Path("csv_data")
    html_output_folder = Path("Html_data")

    # 🔹建立輸出資料夾（若不存在）
    gpx_output_folder.mkdir(exist_ok=True)
    csv_output_folder.mkdir(exist_ok=True)
    html_output_folder.mkdir(exist_ok=True)

    print("--- 開始批次處理 ---")
    print(f"🔍 目前工作目錄: {os.getcwd()}")
    print(f"📂 正在從 '{gpx_folder.resolve()}' 讀取 GPX 檔案")
    print(f"📂 正在從 '{txt_folder.resolve()}' 讀取 TXT 檔案")
    print("-" * 40)

    # 🔹 尋找所有 .gpx 檔案
    gpx_files = list(gpx_folder.glob("*.gpx"))

    if not gpx_files:
        print("❌ 錯誤：在 'hikingbook_route' 資料夾中找不到任何 .gpx 檔案。")
        print("   請確認：")
        print("   1. 'hikingbook_route' 資料夾與您的 Python 腳本位於同一層級。")
        print("   2. 資料夾內確實有 .gpx 結尾的檔案。")
    else:
        print(f"✅ 找到 {len(gpx_files)} 個 GPX 檔案，準備開始處理...\n")

    # 🔹迴圈處理每個 GPX 檔案
    for gpx_file in gpx_files:
        base_name = gpx_file.stem  # 檔名不含副檔名
        txt_file = txt_folder / f"{base_name}.txt"

        print(f"🔸 正在處理: {gpx_file.name}")
        print(f"   - 正在尋找對應的 TXT: {txt_file.name}")

        if not txt_file.exists():
            print(f"   - ⚠️ 警告: 找不到對應的 TXT 檔案，略過此 GPX 檔案。")
            print("-" * 40)
            continue

        print(f"   - ✓ 找到對應的 TXT 檔案。")

        # 1. 去除時間
        gpx_obj = remove_time(str(gpx_file))

        # 2. 插入通訊點
        comm_pts = load_communication_points(str(txt_file))
        if not comm_pts:
            print(f"   - ⚠️ 警告: 未能從 {txt_file.name} 讀取任何通訊點，略過此檔案。")
            print("-" * 40)
            continue

        gpx_with_pts = insert_points(gpx_obj, comm_pts)

        # 3. 切出路段
        start_latlon = (comm_pts[0][1], comm_pts[0][2])
        end_latlon = (comm_pts[-1][1], comm_pts[-1][2])
        sliced_gpx = split_single_segment(gpx_with_pts, start_latlon, end_latlon)

        # 4. 匯出 GPX + CSV
        gpx_outfile = gpx_output_folder / f"{base_name}_processed.gpx"
        csv_outfile = csv_output_folder / f"{base_name}_processed.csv"
        export_gpx_and_csv(sliced_gpx, str(gpx_outfile), str(csv_outfile))

        # 5. 儲存地圖 HTML
        html_outfile = html_output_folder / f"{base_name}_map.html"
        visualize_with_checkbox(gpx_with_pts, sliced_gpx, comm_pts, str(html_outfile))

        print("-" * 40)

    print("\n🏁 批次處理完成")

--- 開始批次處理 ---
🔍 目前工作目錄: c:\資展\專題\特徵
📂 正在從 'C:\資展\專題\特徵\hikingbook_route' 讀取 GPX 檔案
📂 正在從 'C:\資展\專題\特徵\txt_data' 讀取 TXT 檔案
----------------------------------------
✅ 找到 5 個 GPX 檔案，準備開始處理...

🔸 正在處理: chiyou_pintian.gpx
   - 正在尋找對應的 TXT: chiyou_pintian.txt
   - ✓ 找到對應的 TXT 檔案。
    - 已移除時間戳記
    - 成功讀取 7 個通訊點
    - 已將所有通訊點插入軌跡
    - 路線切割完成：從索引 6 至 83
    - ✅ 儲存 GPX：gpx_data\chiyou_pintian_processed.gpx
    - ✅ 匯出 CSV：csv_data\chiyou_pintian_processed.csv
    - ✅ 地圖已儲存：Html_data\chiyou_pintian_map.html
----------------------------------------
🔸 正在處理: tao.gpx
   - 正在尋找對應的 TXT: tao.txt
   - ✓ 找到對應的 TXT 檔案。
    - 已移除時間戳記
    - 成功讀取 4 個通訊點
    - 已將所有通訊點插入軌跡
    - 路線切割完成：從索引 7 至 165
    - ✅ 儲存 GPX：gpx_data\tao_processed.gpx
    - ✅ 匯出 CSV：csv_data\tao_processed.csv
    - ✅ 地圖已儲存：Html_data\tao_map.html
----------------------------------------
🔸 正在處理: tao_kalaye.gpx
   - 正在尋找對應的 TXT: tao_kalaye.txt
   - ✓ 找到對應的 TXT 檔案。
    - 已移除時間戳記
    - 成功讀取 8 個通訊點
    - 已將所有通訊點插入軌跡
    - 路線切割完成：從索引 7 至 182
   

gpt


In [ ]:
import gpxpy
import gpxpy.gpx
import pandas as pd
from pathlib import Path
from typing import List, Tuple
from haversine import haversine, Unit
import folium
from folium.plugins import MarkerCluster
import copy

# ====== STEP 1: 預處理函式 ======


def remove_time(gpx_file: str) -> gpxpy.gpx.GPX:
    with open(gpx_file, "r", encoding="utf-8") as f:
        gpx = gpxpy.parse(f)
    for track in gpx.tracks:
        for segment in track.segments:
            for point in segment.points:
                point.time = None  # 移除時間
    return gpx


def load_communication_points(txt_file: str) -> List[Tuple[str, float, float, float]]:
    df = pd.read_csv(txt_file, sep="\t")
    points = []
    for _, row in df.iterrows():
        name = row["路標指示"]
        lat = float(row["緯度"])
        lon = float(row["經度"])
        ele = float(row["海拔（約）"])
        points.append((name, lat, lon, ele))
    return points


# ====== STEP 2: 插入與修剪邏輯 ======


def find_nearest_index(
    gpx_obj: gpxpy.gpx.GPX, target_lat: float, target_lon: float
) -> int:
    min_dist = float("inf")
    nearest_idx = 0
    points = gpx_obj.tracks[0].segments[0].points
    for idx, pt in enumerate(points):
        dist = haversine(
            (pt.latitude, pt.longitude), (target_lat, target_lon), unit=Unit.METERS
        )
        if dist < min_dist:
            min_dist = dist
            nearest_idx = idx
    return nearest_idx


def trim_and_insert_points(
    gpx_obj: gpxpy.gpx.GPX, comm_pts: List[Tuple[str, float, float, float]]
) -> gpxpy.gpx.GPX:
    last_name, last_lat, last_lon, _ = comm_pts[-1]
    trim_end_idx = find_nearest_index(gpx_obj, last_lat, last_lon)
    points = gpx_obj.tracks[0].segments[0].points
    trimmed_points = points[: trim_end_idx + 1]

    trimmed_gpx = gpxpy.gpx.GPX()
    track = gpxpy.gpx.GPXTrack()
    segment = gpxpy.gpx.GPXTrackSegment()
    segment.points = trimmed_points
    track.segments.append(segment)
    trimmed_gpx.tracks.append(track)

    final_gpx = insert_points(trimmed_gpx, comm_pts)
    return final_gpx


def insert_points(
    gpx_obj: gpxpy.gpx.GPX, comm_pts: List[Tuple[str, float, float, float]]
) -> gpxpy.gpx.GPX:
    segment = gpx_obj.tracks[0].segments[0]
    original_pts = segment.points

    new_points = []
    for name, lat, lon, ele in comm_pts:
        nearest_idx = find_nearest_index(gpx_obj, lat, lon)
        nearby_range = range(
            max(0, nearest_idx - 2), min(len(original_pts), nearest_idx + 3)
        )
        nearby_pts = [(i, original_pts[i]) for i in nearby_range]
        nearby_pts.sort(
            key=lambda x: haversine(
                (x[1].latitude, x[1].longitude), (lat, lon), unit=Unit.METERS
            )
        )
        insert_idx = nearby_pts[0][0]

        comm_point = gpxpy.gpx.GPXTrackPoint(
            latitude=lat, longitude=lon, elevation=ele, name=name
        )
        original_pts.insert(insert_idx, comm_point)
        new_points.append((insert_idx, comm_point))
    return gpx_obj


# ====== STEP 3: 匯出與視覺化 ======


def export_gpx_and_csv(
    gpx_obj: gpxpy.gpx.GPX, output_gpx_path: str, output_csv_path: str
):
    with open(output_gpx_path, "w", encoding="utf-8") as f:
        f.write(gpx_obj.to_xml())

    data = [
        (pt.latitude, pt.longitude, pt.elevation)
        for pt in gpx_obj.tracks[0].segments[0].points
    ]
    df = pd.DataFrame(data, columns=["latitude", "longitude", "elevation"])
    df.to_csv(output_csv_path, index=False)


def visualize_with_checkbox(
    original_gpx: gpxpy.gpx.GPX,
    final_gpx: gpxpy.gpx.GPX,
    comm_pts: List[Tuple[str, float, float, float]],
    html_path: str,
):
    first_point = original_gpx.tracks[0].segments[0].points[0]
    m = folium.Map(
        location=[first_point.latitude, first_point.longitude], zoom_start=14
    )

    def plot_line(gpx_obj, color, name):
        coords = [
            (pt.latitude, pt.longitude) for pt in gpx_obj.tracks[0].segments[0].points
        ]
        folium.PolyLine(
            coords, color=color, weight=4, opacity=0.7, tooltip=name
        ).add_to(m)

    plot_line(original_gpx, "blue", "原始路線")
    plot_line(final_gpx, "red", "處理後路線")

    cluster = MarkerCluster().add_to(m)
    for name, lat, lon, ele in comm_pts:
        folium.Marker([lat, lon], popup=name, icon=folium.Icon(color="purple")).add_to(
            cluster
        )

    m.save(html_path)


# ====== STEP 4: 主程式批次處理 ======

if __name__ == "__main__":
    input_gpx_folder = Path("hikingbook_route")
    input_txt_folder = Path("txt_data")
    output_gpx_folder = Path("gpx_data")
    output_csv_folder = Path("csv_data")
    output_html_folder = Path("Html_data")

    output_gpx_folder.mkdir(exist_ok=True)
    output_csv_folder.mkdir(exist_ok=True)
    output_html_folder.mkdir(exist_ok=True)

    for gpx_file in input_gpx_folder.glob("*.gpx"):
        txt_file = input_txt_folder / f"{gpx_file.stem}.txt"
        if not txt_file.exists():
            print(f"❌ 缺少通訊點 TXT：{txt_file.name}，略過。")
            continue

        print(f"✅ 處理中：{gpx_file.name}")
        gpx_obj = remove_time(str(gpx_file))
        original_gpx_copy = copy.deepcopy(gpx_obj)
        comm_pts = load_communication_points(str(txt_file))

        final_gpx = trim_and_insert_points(gpx_obj, comm_pts)

        output_gpx_path = output_gpx_folder / gpx_file.name
        output_csv_path = output_csv_folder / f"{gpx_file.stem}.csv"
        output_html_path = output_html_folder / f"{gpx_file.stem}.html"

        export_gpx_and_csv(final_gpx, str(output_gpx_path), str(output_csv_path))
        visualize_with_checkbox(
            original_gpx_copy, final_gpx, comm_pts, str(output_html_path)
        )
        print(f"🎉 完成：{gpx_file.stem}")

✅ 處理中：chiyou_pintian.gpx
🎉 完成：chiyou_pintian
✅ 處理中：tao.gpx


ValueError: could not convert string to float: '1400 m'